<div style="text-align: right">CSCI E-7 Introduction to Python Programming for Life Sciences</div>
<div style="text-align: right">Dino Konstantopoulos, 4 March 2019</div>

Let's apply what we learned about RNA genetic machinery and write general programs that borrow this machinery to navigate the state space of a problem in a disciplined manner.

# Guess my number

A simple game for two people where one picks a secret number between 1 and 10 and the other has to guess that number.

Is it 2?  No

Is it 3?  No

Is it 7?  No

Is it 1?  Yes

That works reasonably well for 1..10 but quickly becomes frustrating or boring as we increase the range to 1..100 or 1..1000. Right? Because we have no way to improve our guesses. There’s no challenge. The guess is either right or wrong, so it quickly becomes a mechanical process.

# More interesting

So, to make it more interesting, instead of *no* let’s say **higher** or **lower**.

1?  Higher

7?  Lower

6?  Lower

5?  Lower

4?  Correct

That might be reasonably interesting for a while, for 1..10 maybe, but soon you’ll increase the range to 1..100. Because people are competitive, the next challenge will be to see who is a better guesser by trying to find the number in the *fewest* guesses. At this point the person who evolves the most efficient guessing strategy wins!

# Genetic algorithm

* A genetic algorithm does not know what *lower* means. It has no *intelligence*. It does not learn. It will make the same mistake every time. It will only be as good at solving a problem as the person who wrote the code. 

And yet, it can be used to find solutions to problems that humans would struggle to solve or could not solve at all. How is that possible?

When playing a card game, inexperienced players build a mental map using the cards in their hand and those on the table. 

More experienced players also take advantage of their knowledge of the problem space, the **entire set of cards in the deck**. 

This means they may also keep track of cards that have not yet been played, and may know they can win the rest of the rounds without having to play them out. 

Highly experienced card players also know the probabilities of various winning combinations. They've written out the probability function we started class with!

Professionals, who earn their living playing the game, also pay attention to the way their competitors play.

Genetic algorithms use **random exploration of the problem space** combined with *evolutionary processes* like **mutation** and **mating** or **crossover** (exchange of genetic information) to improve guesses. But also, because they have no experience in the problem domain, they try things a human would never think to try. Thus, a person using a genetic algorithm may learn more about the problem space and potential solutions. This gives them the ability to make improvements to the algorithm.

# Guess the Password

Now let’s see how this all applies to guessing a password. We’ll start by randomly generating an initial sequence of letters and then mutate/change one random letter at a time until the sequence of letters becomes "Hello World!". 

Conceptually:

```python
_letters = [a..zA..Z !]
target = "Hello World!"
guess = get 12 random letters from _letters
while guess != target:
   index = get random value from [0..length of target]
   guess[index] = get 1 random value from _letters
```
If you try this in any programming language you’ll find that it performs worse than playing the number guessing game with only **yes** and **no** answers because it cannot tell when one guess is better than another.

So, let’s help it make an informed guess by telling it how many of the letters from the guess are in the correct locations. For example "World!Hello?" would get 2 because only the 4th letter of each word is correct. The 2 indicates how close the answer is to correct. This is called the **fitness value**. 

"hello world?" would get a fitness value of 9 because 9 letters are correct. Only the `h`, `w`, and question mark are wrong.

# Genes

We start off with a generic set of letters for genes and a target password:

In [1]:
geneSet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
target = "Hello World!"

# Generate a guess

Next we need a way to generate a random string of letters from the gene set.

In [2]:
import random

def generate_parent(length):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

`random.sample` takes sampleSize values from the input **without replacement**. 

This means there will be no duplicates in the generated parent unless geneSet contains duplicates, or length is greater than `len(geneSet)`. The implementation above allows us to generate a long string with a small set of genes while using as many unique genes as possible.

# Fitness

The fitness value the genetic algorithm provides is the only feedback the engine gets to guide it toward a solution. In this problem our fitness value is the total number of letters in the guess that match the letter in the same position of the password.

In [3]:
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target, guess)
               if expected == actual)

# Mutate

We also need a way to produce a new guess by **mutating** the current one. The following implementation converts the parent string to an array with list(parent) then replaces 1 letter in the array with a randomly selected one from geneSet, and then recombines the result into a string with ''.join(genes).

In [4]:
def mutate(parent):
    index = random.randrange(0, len(parent))
    childGenes = list(parent)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = (alternate  
        if newGene == childGenes[index]  
        else newGene)
    return ''.join(childGenes)

This implementation uses an alternate replacement if the randomly selected newGene is the same as the one it is supposed to replace, which can save a significant amount of overhead.

# Display

Next, it is important to monitor what is happening, so that we can stop the engine if it gets stuck. It also allows us to learn what works and what does not so we can improve the algorithm. This is also called **debugging** ;-)

We’ll display a visual representation of the gene sequence, which may not be the literal gene sequence, its fitness value and how much time has elapsed.

In [5]:
import datetime

def display(guess):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(guess)
    print("{0}\t{1}\t{2}".format(guess, fitness, str(timeDiff)))

# Main

Now we’re ready to write the main program. We start by initializing `bestParent` to a random sequence of letters.

In [6]:
random.seed()
startTime = datetime.datetime.now()
bestParent = generate_parent(len(target))
bestFitness = get_fitness(bestParent)
display(bestParent)

qcieUFstlOfE	0	0:00:00.000580


Then we add the heart of the genetic engine. It is a **loop** that generates a guess, requests the fitness for that guess, then compares it to that of the previous best guess, and keeps the better of the two. This cycle repeats until all the letters match those in the target.

In [7]:
while True:
    child = mutate(bestParent)
    childFitness = get_fitness(child)

    if bestFitness >= childFitness:
        continue
    display(child)
    if childFitness >= len(bestParent):
        break
    bestFitness = childFitness
    bestParent = child

qcieUFstlOdE	1	0:00:01.681844
qcieUFstlOd!	2	0:00:01.682238
qcieUFWtlOd!	3	0:00:01.682338
qcleUFWtlOd!	4	0:00:01.682468
qcleoFWtlOd!	5	0:00:01.682586
qcleoFWtrOd!	6	0:00:01.683094
qcleoFWorOd!	7	0:00:01.683261
qeleoFWorOd!	8	0:00:01.687977
qelloFWorOd!	9	0:00:01.690071
HelloFWorOd!	10	0:00:01.695060
Hello WorOd!	11	0:00:01.697130
Hello World!	12	0:00:01.700293


# Extract a reusable engine

Now that we have a working solution to this problem we will extract the genetic engine code from that specific to the password problem so we can reuse it to solve other problems.

We’ll rename the `mutate` and `generate_paren`t functions to _mutate and _generate_parent. This is how protected functions are named in Python. They will not be visible to users of the genetic library iof they lives in a python file.

### Generate and Mutate

Since we want to be able to customize the gene set used in future problems we need to pass it as a parameter to `_generate_parent`

In [8]:
import random

def _generate_parent(length, geneSet):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

In [9]:
def _mutate(parent, geneSet):
    index = random.randrange(0, len(parent))
    childGenes = list(parent)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = (alternate  
        if newGene == childGenes[index]  
        else newGene)
    return ''.join(childGenes)

### get_best

Next we’ll move the main loop into a new function named `get_best` in our genetic library. 

Its parameters will include the functions it should use to request the fitness for a guess and to display (or report) each new best guess as it is found, the number of genes to use when creating a new sequence, the optimal fitness, and the set of genes to use for creating and mutating gene sequences.

In [10]:
def get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
    random.seed()
    bestParent = _generate_parent(targetLen, geneSet)
    bestFitness = get_fitness(bestParent)
    display(bestParent)
    if bestFitness >= optimalFitness:
        return bestParent

    while True:
        child = _mutate(bestParent, geneSet)
        childFitness = get_fitness(child)

        if bestFitness >= childFitness:
            continue
        display(child)
        if childFitness >= optimalFitness:
            return child
        bestFitness = childFitness
        bestParent = child

Notice that we call` display` and `get_fitness` with only one parameter - the child gene sequence. This is because we do not want the engine to have access to the target value, and it doesn’t care whether we are timing the run or not, so those are not passed to the function.

We now have a reusable library that we can play with.

In [11]:
def test_Hello_World():
    #target = "Hello World!"
    target = "For I am fearless and made of chocolate."
    guess_password(target)


def guess_password(target):
    geneset = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
    startTime = datetime.datetime.now()

    def fnGetFitness(genes):
        return get_fitness(genes, target)

    def fnDisplay(genes):
        display(genes, target, startTime)

    optimalFitness = len(target)
    get_best(fnGetFitness, len(target), optimalFitness, geneset, fnDisplay)

In [12]:
def get_fitness(genes, target):
    return sum(1 for expected, actual in zip(target, genes)
               if expected == actual)

In [13]:
def display(genes, target, startTime):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(genes, target)
    print("{0}\t{1}\t{2}".format(genes, fitness, str(timeDiff)))

In [14]:
test_Hello_World()

vtjAZUuF VJ.k!SwhQcPGfLzEXWKNxyYeIlRCdmB	1	0:00:00.000509
vtjAZUuF VJ.k!SwhQcPGfLzEXWKfxyYeIlRCdmB	2	0:00:00.001586
vtjAZUuF Ve.k!SwhQcPGfLzEXWKfxyYeIlRCdmB	3	0:00:00.005526
vtjAZUum Ve.k!SwhQcPGfLzEXWKfxyYeIlRCdmB	4	0:00:00.007662
vtjAZUum Ve.k!SwhQcPG LzEXWKfxyYeIlRCdmB	5	0:00:00.008443
vtjAZUum Ve.k!ewhQcPG LzEXWKfxyYeIlRCdmB	6	0:00:00.008708
vtjAZUum Ve.k!ewsQcPG LzEXWKfxyYeIlRCdmB	7	0:00:00.010401
vtjAZUum Ve.k!ewsQcPG LzEeWKfxyYeIlRCdmB	8	0:00:00.011646
vtjAZUum Ve.k!ewsQcPG LzEeWKfxcYeIlRCdmB	9	0:00:00.012168
vtjAZUum Ve.k!ewsQcPG LzdeWKfxcYeIlRCdmB	10	0:00:00.012439
vtjAZUum Ve.k!ewsQcPG LzdeWKfxcYeclRCdmB	11	0:00:00.014302
vtjAZUum Ve.k!ewsQcPG LzdeWKf cYeclRCdmB	12	0:00:00.016262
vtjAZUum Ve.k!ewsQcPG LzdeWKf cYecoRCdmB	13	0:00:00.017350
vtjAIUum Ve.k!ewsQcPG LzdeWKf cYecoRCdmB	14	0:00:00.023340
vtjAI um Ve.k!ewsQcPG LzdeWKf cYecoRCdmB	15	0:00:00.024332
vtjAI um Ve.k!ewsQcPG mzdeWKf cYecoRCdmB	16	0:00:00.026069
vtjAI um Ve.k!ewsQcPG mzde Kf cYecoRCdmB	17	0:00:00.027307
vojAI 

# A bit of OOP

Let's group all that functionality in nice OOP form.

Here, we declare our `Chromosome` class and some built-in utility functions (`_xxx`) that, if we place in a python source code file, should not be callable from third party sources.

In [39]:
import random
import statistics
import sys
import time

class Chromosome:
    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness
        

def _generate_parent(length, geneSet, get_fitness):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    #genes = ''.join(genes)
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)


def _mutate(parent, geneSet, get_fitness):
    index = random.randrange(0, len(parent.Genes))
    childGenes = list(parent.Genes)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    #genes = ''.join(childGenes)
    genes = childGenes #Keep genes as a list
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)


def _get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
    random.seed()
    bestParent = _generate_parent(targetLen, geneSet, get_fitness)
    display(bestParent)
    if bestParent.Fitness > optimalFitness or bestParent.Fitness.NumbersInSequenceCount == optimalFitness.NumbersInSequenceCount:
        return bestParent
    while True:
        child = _mutate(bestParent, geneSet, get_fitness)
        if bestParent.Fitness > child.Fitness or bestParent.Fitness.NumbersInSequenceCount == optimalFitness.NumbersInSequenceCount:
            continue
        display(child)
        if child.Fitness.NumbersInSequenceCount == optimalFitness.NumbersInSequenceCount:
            break
        elif child.Fitness > optimalFitness:
            return child
        bestParent = child

Here we define our fitness and display (debugging) functions, and also the class `GuessPasswordTests` that will solve the problem at hand: Guessing passwords. Notice how we define functions within functions, which lets us easily swap different options in order to test our program.

In [135]:
def get_fitness(guess, target):
    return sum(1 for expected, actual in zip(target, guess)
               if expected == actual)


def display(candidate, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}\t{}\t{}".format(
        candidate.Genes, candidate.Fitness, timeDiff))


class GuessPasswordTests():
    geneset = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!.,"

    def test_Hello_World(self):
        target = "Hello World!"
        self.guess_password(target)

    def test_For_I_am_fearless_and_made_of_chocolate(self):
        target = "For I am fearless and made of chocolate."
        self.guess_password(target)

    def guess_password(self, target):
        startTime = datetime.datetime.now()

        def fnGetFitness(genes):
            return get_fitness(genes, target)

        def fnDisplay(candidate):
            display(candidate, startTime)

        optimalFitness = len(target)
        best = _get_best(fnGetFitness, len(target), optimalFitness,
                                self.geneset, fnDisplay)

    def test_Random(self):
        length = 150
        target = ''.join(random.choice(self.geneset)
                         for _ in range(length))

        self.guess_password(target)

Let's create an object of the class `GuessPasswordTests`, and call its `test_For_I_am_fearless_and_made_of_chocolate()` method.

In [45]:
t = GuessPasswordTests()
t.test_For_I_am_fearless_and_made_of_chocolate()

# Homework

<b>Q1:</b> Use `GuessPasswordTests` as a model to build a genetic algorithm that sorts lists of integers, and compare its performance to that of bubble sort and merge sort, which we learned about in this lecture. Run this class on a random list of 10 integers. You will need to define your own fitness function, but I think one whereby it counts all *already sorted* elements in the list sounds pretty good to me! We will look for your use of recursion/iteration as needed, class objects as needed and functions. You can start with the function below and modify. <p><b>Points: 30</b></p>

In [40]:
#Define get_fitness and display functions (from example)
def get_fitness(genes):
    fitness = 1
    gap = 0

    for i in range(1, len(genes)):
        if genes[i] > genes[i - 1]:
            fitness += 1
        else:
            gap += genes[i - 1] - genes[i]
    return Fitness(fitness, gap)

def display(candidate, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}\t{}\t{}".format(
        candidate.Genes, candidate.Fitness, timeDiff))

#Define Sorting Class
class TestSortingUnsortedLists():
    geneset = list(range(0, 100))
    
    def test_10_numbers(self):
        self.sort_numbers(10)
        
    def sort_numbers(self, size_of_list):
        startTime = datetime.datetime.now()

        def fnGetFitness(genes):
            return get_fitness(genes)

        def fnDisplay(candidate):
            display(candidate, startTime)
        
        optimalFitness = Fitness(size_of_list, 0)
        best = _get_best(fnGetFitness, size_of_list, optimalFitness,
                                self.geneset, fnDisplay)

#Define Fitness Class
class Fitness:
    def __init__(self, numbersInSequenceCount, totalGap):
        self.NumbersInSequenceCount = numbersInSequenceCount
        self.TotalGap = totalGap

    def __gt__(self, other):
        if self.NumbersInSequenceCount != other.NumbersInSequenceCount:
            return self.NumbersInSequenceCount > other.NumbersInSequenceCount
        return self.TotalGap < other.TotalGap

    def __str__(self):
        return "{} Sequential, {} Total Gap".format(
            self.NumbersInSequenceCount,
            self.TotalGap)

In [157]:
geneset = list(range(0, 10))
#', '.join(str(v) for v in geneset)
geneset[9]

9

In [68]:
j = TestSortingUnsortedLists()
#j.test_10_numbers()

In [69]:
j.test_10_numbers()

[70, 28, 5, 9, 59, 47, 38, 34, 56, 82]	5 Sequential, 90 Total Gap	0:00:00.000337
[70, 28, 5, 9, 59, 47, 38, 42, 56, 82]	6 Sequential, 86 Total Gap	0:00:00.000723
[70, 28, 5, 9, 59, 95, 38, 42, 56, 82]	7 Sequential, 122 Total Gap	0:00:00.000873
[70, 28, 5, 9, 59, 13, 38, 42, 56, 82]	7 Sequential, 111 Total Gap	0:00:00.000990
[70, 30, 5, 9, 59, 13, 38, 42, 56, 82]	7 Sequential, 111 Total Gap	0:00:00.001076
[70, 30, 5, 9, 19, 13, 38, 42, 56, 82]	7 Sequential, 71 Total Gap	0:00:00.001161
[2, 30, 5, 9, 19, 13, 38, 42, 56, 82]	8 Sequential, 31 Total Gap	0:00:00.001263
[2, 30, 5, 9, 19, 13, 33, 42, 56, 82]	8 Sequential, 31 Total Gap	0:00:00.001372
[2, 30, 5, 9, 19, 22, 33, 42, 56, 82]	9 Sequential, 25 Total Gap	0:00:00.001618
[2, 30, 5, 9, 19, 22, 33, 42, 56, 59]	9 Sequential, 25 Total Gap	0:00:00.001783
[9, 30, 5, 9, 19, 22, 33, 42, 56, 59]	9 Sequential, 25 Total Gap	0:00:00.002026
[9, 3, 5, 9, 19, 22, 33, 42, 56, 59]	9 Sequential, 6 Total Gap	0:00:00.002118
[9, 3, 5, 9, 19, 32, 33, 42, 56, 

## For example

This might be an example of a solution run:

```python
t = TestSortingUnsortedLists()
t.test_please_sort_10_numbers()
```

```python
95, 36, 90, 94, 7, 52, 21, 81, 99, 91	=> 6 Sequential, 185 Total Gap	0:00:00
95, 65, 90, 94, 7, 52, 21, 32, 99, 91	=> 6 Sequential, 156 Total Gap	0:00:00.000977
26, 65, 90, 94, 7, 52, 21, 32, 99, 91	=> 7 Sequential, 126 Total Gap	0:00:00.000977
35, 65, 90, 94, 7, 52, 21, 75, 98, 91	=> 7 Sequential, 125 Total Gap	0:00:00.000977
8, 65, 90, 94, 7, 52, 21, 68, 92, 91	=> 7 Sequential, 119 Total Gap	0:00:00.000977
8, 65, 90, 94, 7, 52, 35, 68, 92, 91	=> 7 Sequential, 105 Total Gap	0:00:00.000977
8, 65, 90, 94, 7, 52, 80, 68, 92, 91	=> 7 Sequential, 100 Total Gap	0:00:00.000977
8, 65, 90, 94, 43, 52, 80, 68, 92, 91	=> 7 Sequential, 64 Total Gap	0:00:00.000977
8, 20, 90, 94, 43, 52, 47, 68, 92, 91	=> 7 Sequential, 57 Total Gap	0:00:00.000977
4, 30, 90, 94, 43, 52, 47, 68, 79, 91	=> 8 Sequential, 56 Total Gap	0:00:00.000977
12, 58, 90, 94, 43, 52, 77, 76, 85, 88	=> 8 Sequential, 52 Total Gap	0:00:00.000977
12, 58, 90, 94, 43, 52, 66, 76, 85, 88	=> 9 Sequential, 51 Total Gap	0:00:00.001964
12, 15, 23, 48, 43, 45, 75, 77, 85, 88	=> 9 Sequential, 5 Total Gap	0:00:00.001964
17, 18, 25, 30, 43, 55, 62, 77, 83, 91	=> 10 Sequential, 0 Total Gap	0:00:00.003990
```

where our fitness function is:
```python
def get_fitness(genes):
    fitness = 1
    gap = 0

    for i in range(1, len(genes)):
        if genes[i] > genes[i - 1]:
            fitness += 1
        else:
            gap += genes[i - 1] - genes[i]
    return Fitness(fitness, gap)
```

which uses the class:
```python

class Fitness:
    def __init__(self, numbersInSequenceCount, totalGap):
        self.NumbersInSequenceCount = numbersInSequenceCount
        self.TotalGap = totalGap

    def __gt__(self, other):
        if self.NumbersInSequenceCount != other.NumbersInSequenceCount:
            return self.NumbersInSequenceCount > other.NumbersInSequenceCount
        return self.TotalGap < other.TotalGap

    def __str__(self):
        return "{} Sequential, {} Total Gap".format(
            self.NumbersInSequenceCount,
            self.TotalGap)
```

Isn't it amazing how we can teach a program to do *anything*, using the machinery of life?

# Merge Sort and Bubble Sort

In [19]:
#Create a Bubble Sort function (from lecture)
def bubble_sort(alist):
    for passnum in range(len(alist)-1, 0, -1):
        for i in range(passnum):
            if alist[i] > alist[i+1]:
                temp = alist[i]
                alist[i] = alist[i+1]
                alist[i+1] = temp
                
#Create a Merge Sort function (from lecture)
def merge_sort(alist):
    #print("Splitting ", alist)
    
    if len(alist) > 1:
        mid = len(alist)//2
        lefthalf = alist[:mid]
        righthalf = alist[mid:]
        
        merge_sort(lefthalf)
        merge_sort(righthalf)
        
        i = 0
        j = 0
        k = 0
        
        while i < len(lefthalf) and j < len(righthalf):
            if lefthalf[i] < righthalf[j]:
                alist[k] = lefthalf[i]
                i += 1
            else:
                alist[k] = righthalf[j]
                j += 1
                
            k += 1
                
        while i < len(lefthalf):
            alist[k] = lefthalf[i]
            i += 1
            k += 1
            
        while j < len(righthalf):
            alist[k] = righthalf[j]
            j += 1
            k += 1

In [70]:
genes = [70, 28, 5, 9, 59, 47, 38, 34, 56, 82]

In [71]:
%%time
bubble_sort(genes)

CPU times: user 41 µs, sys: 24 µs, total: 65 µs
Wall time: 86.8 µs


In [72]:
%%time
merge_sort(genes)

CPU times: user 52 µs, sys: 0 ns, total: 52 µs
Wall time: 58.2 µs


Overall, it seems that "mutate" sorting is slower than both the bubble sort and merge sort. The mutate function takes  3441 µs to created a sorted list from [70, 28, 5, 9, 59, 47, 38, 34, 56, 82] while bubble sort and merge sort order the internal lists and take far less time (86.8 µs and 58.2 µs respectively).